In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_columns', None)

In [2]:
ROOT = "/media/school/project/data.census.gov_downloads"

In [3]:
# Read 2018 gaz
gaz = pd.read_csv(os.path.join(ROOT, '2018_Gaz_tracts_national.txt'), sep='\t')
print(gaz.info())
gaz.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 8 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   USPS                                                                                                                                    74001 non-null  object 
 1   GEOID                                                                                                                                   74001 non-null  int64  
 2   ALAND                                                                                                                                   74001 non-null  int64  
 3   AWATER                                                        

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9817813,28435,3.791,0.011,32.481959,-86.491338
1,AL,1001020200,3325679,5669,1.284,0.002,32.475758,-86.472468
2,AL,1001020300,5349273,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6384276,8408,2.465,0.003,32.471030,-86.444835
4,AL,1001020500,11408869,43534,4.405,0.017,32.458922,-86.421826


In [4]:
# iterate over downloaded data and build dfs from the csvs
# dataframes stored in dfs dict which maps name: df
dfs = {}
for x in os.listdir(ROOT):
    x_path = os.path.join(ROOT, x)
    if not os.path.isdir(x_path):
        continue
    print(x)
    for y in os.listdir(x_path):
        y_path = os.path.join(x_path, y)
        if not os.path.isdir(y_path):
            continue
        print('----'+y)
        for z in os.listdir(y_path):
            z_path = os.path.join(y_path, z)
            if not z.endswith('csv') or 'meta' in z:
                continue
            print('--------' + z)
            dfs[x] = pd.read_csv(z_path, low_memory=False)

race
----ACSDT5Y2018.B02001_2020-10-11T113117
--------ACSDT5Y2018.B02001_data_with_overlays_2020-10-11T112948.csv
acs_demographic_and_housing_estimates
----ACSDP5Y2018.DP05_2020-10-11T112022
--------ACSDP5Y2018.DP05_data_with_overlays_2020-10-11T111820.csv
employment_status
----ACSST5Y2018.S2301_2020-10-11T112308
--------ACSST5Y2018.S2301_data_with_overlays_2020-10-11T112123.csv
educational_attainment
----ACSST5Y2018.S1501_2020-10-11T112704
--------ACSST5Y2018.S1501_data_with_overlays_2020-10-11T112246.csv
.ipynb_checkpoints
age_and_sex
----ACSST5Y2018.S0101_2020-10-11T112952
--------ACSST5Y2018.S0101_data_with_overlays_2020-10-11T112700.csv
households_and_families
----ACSST5Y2018.S1101_2020-10-11T113434
--------ACSST5Y2018.S1101_data_with_overlays_2020-10-11T113113.csv
income_and_poverty
----ACSST5Y2018.S1701_2020-10-11T110717
--------ACSST5Y2018.S1701_data_with_overlays_2020-10-11T110149.csv
industry_by_occupation
----ACSST5Y2018.S2405_2020-10-11T113601
--------ACSST5Y2018.S2405_data

In [5]:
dfs.keys()

dict_keys(['race', 'acs_demographic_and_housing_estimates', 'employment_status', 'educational_attainment', 'age_and_sex', 'households_and_families', 'income_and_poverty', 'industry_by_occupation'])

In [6]:
def prepare_df(df):
    """
    prepare data.census.gov downloads
    returns prepared df and data dictionary df
    """
    # remove columns that end with M (remove margin of error fields)
    n_cols_before = df.shape[1]
    df = df[[x for x in df.columns if not x.endswith('M')]]
    n_cols_after = df.shape[1]
    assert n_cols_after == n_cols_before / 2 + 1 # make sure we arent dropping anything unexpected
    #display(df.head(3))
    
    # create data dict from columns and first row
    data_dict = df.iloc[0].reset_index()
    data_dict.columns = ['column', 'description']
    #display(data_dict)
    
    # remove first row with descriptions and last row with total US summary data
    df = df.iloc[1:-1]
    #display(df.head(3))
    
   
    # create GEOID as the tail portion of GEO_ID
    df['GEOID'] = df['GEO_ID'].str.split('US').str[-1]
    df.GEOID = pd.to_numeric(df.GEOID, errors='coerce', downcast='integer')
    data_dict = pd.concat([pd.DataFrame({'column': ['GEOID'], 'description': ['Field to Join Gazetteer Data']}), data_dict]).reset_index(drop=True)
    
    if df[df.GEOID.isnull()].shape[0] > 1: # 1 will be made for "US" record, that is ok
        print('WARNING: GEOID parsing created {} nulls.'.format(df[df.GEOID.isnull()].shape[0]))
        
    data_cols = list(set(df.columns)-set(['GEOID', 'GEO_ID', 'NAME']))
    
    # convert data fields to numeric type
    #for col in data_cols:
    #    df[col] = pd.to_numeric(df[col], errors='coerce')
        
    # reorder columns
    df = df[['GEOID', 'GEO_ID', 'NAME']+data_cols]
    
    return df, data_dict

In [7]:
data_dicts = {}
for name, df in dfs.items():
    print(name)
    dfs[name], data_dicts[name] = prepare_df(df)

race
acs_demographic_and_housing_estimates
employment_status
educational_attainment
age_and_sex
households_and_families
income_and_poverty
industry_by_occupation


In [8]:
display(dfs['race'].head(2))
display(data_dicts['race'])

,GEOID,GEO_ID,NAME,B02001_002E,B02001_010E,B02001_008E,B02001_005E,B02001_003E,B02001_009E,B02001_006E,B02001_004E,B02001_001E,B02001_007E
1,10003000400,1400000US10003000400,"Census Tract 4, New Castle County, Delaware",1372,40,40,0,1495,0,0,26,2933,0
2,10003001300,1400000US10003001300,"Census Tract 13, New Castle County, Delaware",3223,52,52,110,127,0,0,0,3512,0


,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
4,B02001_002E,Estimate!!Total!!White alone
5,B02001_003E,Estimate!!Total!!Black or African American alone
6,B02001_004E,Estimate!!Total!!American Indian and Alaska Na...
7,B02001_005E,Estimate!!Total!!Asian alone
8,B02001_006E,Estimate!!Total!!Native Hawaiian and Other Pac...
9,B02001_007E,Estimate!!Total!!Some other race alone


In [9]:
# concat data dicts
DD = pd.concat(list(data_dicts.values())).drop_duplicates(keep='first').reset_index(drop=True)

DD

,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
4,B02001_002E,Estimate!!Total!!White alone
...,...,...
1311,S2405_C03_006E,Estimate!!Service occupations!!Civilian employ...
1312,S2405_C04_006E,Estimate!!Sales and office occupations!!Civili...
1313,S2405_C05_006E,"Estimate!!Natural resources, construction, and..."
1314,S2405_C06_006E,"Estimate!!Production, transportation, and mate..."


In [10]:
# join tables
EST = None
for name, table in dfs.items():
    print(name)
    if EST is None:
        EST = table
        continue
    
    print("Before: {}".format(EST.shape))
    EST = EST.merge(table, on=['NAME', 'GEO_ID', 'GEOID'], how='left')    
    print("After: {}".format(EST.shape)) 

race
acs_demographic_and_housing_estimates
Before: (74001, 13)
After: (74001, 191)
employment_status
Before: (74001, 191)
After: (74001, 331)
educational_attainment
Before: (74001, 331)
After: (74001, 715)
age_and_sex
Before: (74001, 715)
After: (74001, 943)
households_and_families
Before: (74001, 943)
After: (74001, 1043)
income_and_poverty
Before: (74001, 1043)
After: (74001, 1226)
industry_by_occupation
Before: (74001, 1226)
After: (74001, 1316)


In [11]:
EST

GEOID                GEO_ID  \
0      10003000400  1400000US10003000400   
1      10003001300  1400000US10003001300   
2      10003002600  1400000US10003002600   
3      10003010200  1400000US10003010200   
4      10003010300  1400000US10003010300   
...            ...                   ...   
73996  53063010304  1400000US53063010304   
73997  53063010501  1400000US53063010501   
73998  53063010601  1400000US53063010601   
73999  53073000700  1400000US53073000700   
74000  53073001100  1400000US53073001100   

                                                  NAME B02001_002E  \
0          Census Tract 4, New Castle County, Delaware        1372   
1         Census Tract 13, New Castle County, Delaware        3223   
2         Census Tract 26, New Castle County, Delaware        1036   
3        Census Tract 102, New Castle County, Delaware        1530   
4        Census Tract 103, New Castle County, Delaware        2138   
...                                                ...         ...   
73996  Census Tract 103.04, Spokane County, Washington        5753   
73997  Census Tract 105.01, Spokane County, Washington        7782   
73998  Census Tract 106.01, Spokane County, Washington        3483   
73999       Census Tract 7, Whatcom County, Washington        4847   
74000      Census Tract 11, Whatcom County, Washington        6380   

      B02001_010E B02001_008E B02001_005E B02001_003E B02001_009E B02001_006E  \
0              40          40           0        1495           0           0   
1              52          52         110         127           0           0   
2              64          64          38        2150           0           0   
3              22          27          34         533           5           0   
4               0           0         253         829           0           0   
...           ...         ...         ...         ...         ...         ...   
73996          65          65          39           0           0           0   
73997         271         271         219           5           0           0   
73998         217         278          41          47          61           0   
73999         301         360         552         199          59           0   
74000         172         226         298          16          54          12   

      B02001_004E B02001_001E B02001_007E DP05_0033PE DP05_0063PE DP05_0085E  \
0              26        2933           0        2933        2933         39   
1               0        3512           0        3512        3512         52   
2               0        3512         224        3512        3512         64   
3               0        2157          33        2157        2157         22   
4               0        3220           0        3220        3220          0   
...           ...         ...         ...         ...         ...        ...   
73996           0        5857           0        5857        5857         65   
73997         222        8554          55        8554        8554        271   
73998          48        3921          24        3921        3921        143   
73999          69        6581         554        6581        6581        262   
74000           0        6957          25        6957        6957        172   

      DP05_0006E DP05_0079PE DP05_0088E DP05_0047PE DP05_0022PE DP05_0028E  \
0             53         0.9       1159         0.0        82.3       83.7   
1             99         0.0       1228         0.0        80.5       79.7   
2            275         0.0        932         0.0        71.0       78.0   
3             53         0.0        868         0.0        75.3      109.6   
4            240         0.0       1035         0.0        75.4       78.7   
...          ...         ...        ...         ...         ...        ...   
73996        372         0.0       2237         0.2        78.3       95.8   
73997        543         1.1       2856         0.0        70.6       90.6   
73998  

In [12]:
### limit to columns that r/a selected from their review of the v3 dataset
selected_df = pd.read_excel(os.path.join(ROOT, 'selected columns_10_24_20.xlsx'))
selected_df

,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
4,DP05_0033PE,Percent Estimate!!RACE!!Total population
...,...,...
284,DP05_0014E,Estimate!!SEX AND AGE!!Total population!!60 to...
285,DP05_0015E,Estimate!!SEX AND AGE!!Total population!!65 to...
286,DP05_0016E,Estimate!!SEX AND AGE!!Total population!!75 to...
287,DP05_0017E,Estimate!!SEX AND AGE!!Total population!!85 ye...


In [13]:
### continued
EST = EST[selected_df['column'].values.tolist()]
EST

,GEOID,GEO_ID,NAME,B02001_001E,DP05_0033PE,DP05_0034PE,DP05_0035PE,DP05_0036PE,DP05_0037PE,DP05_0038PE,DP05_0039PE,DP05_0052PE,DP05_0057PE,DP05_0058PE,DP05_0059PE,DP05_0060PE,DP05_0061PE,DP05_0062PE,DP05_0070PE,DP05_0071PE,DP05_0087PE,DP05_0088PE,DP05_0089PE,DP05_0005PE,DP05_0006PE,DP05_0007PE,DP05_0008PE,DP05_0009PE,DP05_0010PE,DP05_0011PE,DP05_0012PE,DP05_0013PE,DP05_0014PE,DP05_0015PE,DP05_0016PE,DP05_0017PE,S2301_C01_001E,S2301_C02_001E,S2301_C03_001E,S2301_C04_001E,S2301_C01_021E,S2301_C02_021E,S2301_C03_021E,S2301_C04_021E,S1501_C02_055E,S1501_C02_056E,S1501_C02_057E,S1501_C02_058E,S1501_C01_059E,S1501_C03_059E,S1501_C05_059E,S1501_C01_060E,S1501_C03_060E,S1501_C05_060E,S1501_C01_061E,S1501_C03_061E,S1501_C05_061E,S1501_C01_062E,S1501_C03_062E,S1501_C05_062E,S1501_C01_063E,S1501_C03_063E,S1501_C05_063E,S1501_C01_064E,S1501_C03_064E,S1501_C05_064E,S1501_C01_001E,S1501_C03_001E,S1501_C05_001E,S1501_C01_002E,S1501_C03_002E,S1501_C05_002E,S1501_C01_003E,S1501_C03_003E,S1501_C05_003E,S1501_C01_004E,S1501_C03_004E,S1501_C05_004E,S1501_C01_005E,S1501_C04_005E,S1501_C06_005E,S1501_C01_006E,S1501_C03_006E,S1501_C05_006E,S1501_C01_007E,S1501_C03_007E,S1501_C05_007E,S1501_C01_008E,S1501_C03_008E,S1501_C05_008E,S1501_C01_009E,S1501_C03_009E,S1501_C05_009E,S1501_C01_010E,S1501_C03_010E,S1501_C05_010E,S1501_C01_011E,S1501_C03_011E,S1501_C05_011E,S1501_C01_012E,S1501_C03_012E,S1501_C05_012E,S1501_C01_013E,S1501_C03_013E,S1501_C05_013E,S1501_C01_014E,S1501_C04_014E,S1501_C01_015E,S1501_C04_015E,S1501_C01_016E,S1501_C03_016E,S1501_C05_016E,S1501_C01_017E,S1501_C02_017E,S1501_C03_017E,S1501_C04_017E,S1501_C05_017E,S1501_C06_017E,S1501_C01_018E,S1501_C02_018E,S1501_C03_018E,S1501_C04_018E,S1501_C05_018E,S1501_C06_018E,S1501_C01_019E,S1501_C03_019E,S1501_C05_019E,S1501_C01_020E,S1501_C02_020E,S0101_C01_001E,S0101_C03_001E,S0101_C05_001E,S0101_C01_002E,S0101_C03_002E,S0101_C05_002E,S0101_C01_003E,S0101_C03_003E,S0101_C05_003E,S0101_C01_004E,S0101_C03_004E,S0101_C05_004E,S0101_C01_005E,S0101_C03_005E,S0101_C05_005E,S0101_C01_006E,S0101_C03_006E,S0101_C05_006E,S0101_C01_007E,S0101_C03_007E,S0101_C05_007E,S0101_C01_008E,S0101_C03_008E,S0101_C05_008E,S0101_C01_009E,S0101_C03_009E,S0101_C05_009E,S0101_C01_010E,S0101_C03_010E,S0101_C05_010E,S0101_C01_011E,S0101_C03_011E,S0101_C05_011E,S0101_C01_012E,S0101_C03_012E,S0101_C05_012E,S0101_C01_013E,S0101_C03_013E,S0101_C05_013E,S0101_C01_014E,S0101_C03_014E,S0101_C05_014E,S0101_C01_015E,S0101_C03_015E,S0101_C05_015E,S0101_C01_016E,S0101_C03_016E,S0101_C05_016E,S0101_C01_017E,S0101_C03_017E,S0101_C05_017E,S0101_C01_018E,S0101_C03_018E,S0101_C05_018E,S0101_C01_019E,S0101_C03_019E,S0101_C05_019E,S0101_C01_032E,S0101_C03_032E,S0101_C05_032E,S0101_C01_034E,S0101_C01_035E,S0101_C01_036E,S1101_C01_001E,S1101_C02_001E,S1101_C03_001E,S1101_C04_001E,S1101_C05_001E,S1101_C01_002E,S1101_C01_003E,S1101_C01_004E,S1101_C05_009E,S1101_C01_010E,S1101_C02_010E,S1101_C03_010E,S1101_C04_010E,S1101_C05_010E,S1101_C01_011E,S1101_C02_011E,S1101_C03_011E,S1101_C04_011E,S1101_C05_011E,S1101_C01_012E,S1101_C05_012E,S1101_C01_013E,S1101_C05_013E,S1101_C01_014E,S1101_C01_015E,S1101_C01_016E,S1101_C01_017E,S1101_C01_018E,S1101_C01_019E,S1101_C01_020E,S1701_C03_001E,S1701_C03_002E,S1701_C03_003E,S1701_C03_006E,S1701_C03_009E,S1701_C03_010E,S1701_C03_011E,S1701_C03_012E,S2405_C01_008E,S2405_C01_009E,S2405_C01_010E,S2405_C01_011E,S2405_C01_012E,S2405_C01_013E,S2405_C01_014E,S2405_C01_015E,S2405_C01_001E,S2405_C02_001E,S2405_C03_001E,S2405_C04_001E,S2405_C05_001E,S2405_C06_001E,S2405_C01_002E,S2405_C01_003E,S2405_C01_004E,S2405_C01_005E,S2405_C01_006E,S2405_C01_007E,DP05_0033E,DP05_0034E,DP05_0035E,DP05_0036E,DP05_0037E,DP05_0038E,DP05_0039E,DP05_0044E,DP05_0052E,DP05_0057E,DP05_0058E,DP05_0059E,DP05_0060E,DP05_0061E,DP05_0062E,DP05_0070E,DP05_0071E,DP05_0086E,DP05_0087E,DP05_0088E,DP05_0089E,DP05_0001E,DP05_0002E,DP05_0003E,DP05_0004E,DP05_0005E,DP05_0006E,DP05_0007E,DP05_0008E,DP05_0009E,DP05_0010E,DP05_0011E,DP05_0012E,DP05_0013E,D

In [14]:
# Convert relevant fields to a numeric type. Explore data that does not
# easily convert to numeric.
coerced_cnt = 0
total_obj_cols = 0
non_coerced_cols = []
for col in EST:
    if EST.dtypes[col] == 'object' and col not in ['GEO_ID', 'NAME']:
        total_obj_cols += 1
        try:
            new_col = pd.to_numeric(EST[col])
            EST[col] = new_col
            coerced_cnt += 1
        except:
            non_coerced_cols.append(col)
print('{} total object columns examined in this step'.format(total_obj_cols))
print('{} columns successfully converted to a numeric type.'.format(coerced_cnt))
print('{} columns could not be converted in this step to a numeric type.'.format(len(non_coerced_cols)))

<ipython-input-14-721ac8f02be0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EST[col] = new_col


286 total object columns examined in this step
173 columns successfully converted to a numeric type.
113 columns could not be converted in this step to a numeric type.


In [15]:
# perform another prep step to see if we can fillna('0')/replace '', '-' and with '0' and then coerce
# If we can, we WON'T do that replacement with 0s but instead coerce so that missing data is null
step2_coerced_cnt = 0
step2_total_obj_cols = 0
step2_non_coerced_cols = []
for col in EST:
    if EST.dtypes[col] == 'object' and col not in ['GEO_ID', 'NAME']:
        step2_total_obj_cols += 1
        try:
            validation = pd.to_numeric(
                EST[col]\
                    .str.strip()\
                    .str.replace("''", '0')\
                    .str.replace('-', '0')\
                    .fillna('0')
            ) # validation step will fail if there is remaining data not converted to 0
            new_col = pd.to_numeric(EST[col], errors='coerce')
            EST[col] = new_col
            step2_coerced_cnt += 1
        except Exception as e:
            print(col, e)
            step2_non_coerced_cols.append(col)
print('{} total object columns examined in this step'.format(step2_total_obj_cols))
print('{} additional columns converted to a numeric type.'.format(step2_coerced_cnt))
print('{} columns could not be converted in this step to a numeric type.'.format(len(step2_non_coerced_cols)))

<ipython-input-15-21c3fde8fa7a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EST[col] = new_col


S1501_C02_055E Unable to parse string "(X)" at position 13589
S1501_C02_056E Unable to parse string "(X)" at position 13589
S1501_C02_057E Unable to parse string "(X)" at position 13589
S1501_C02_058E Unable to parse string "(X)" at position 13589
S1501_C01_059E Unable to parse string "2,5000" at position 8731
S1501_C03_059E Unable to parse string "2,5000" at position 240
S1501_C05_059E Unable to parse string "2,5000" at position 5471
S1501_C01_060E Unable to parse string "2,5000" at position 18
S1501_C03_060E Unable to parse string "2,5000" at position 0
S1501_C05_060E Unable to parse string "2,5000" at position 18
S1501_C01_061E Unable to parse string "2,5000" at position 240
S1501_C03_061E Unable to parse string "2,5000" at position 216
S1501_C05_061E Unable to parse string "2,5000" at position 17
S1501_C01_062E Unable to parse string "2,5000" at position 240
S1501_C03_062E Unable to parse string "2,5000" at position 240
S1501_C05_062E Unable to parse string "2,5000" at position 240

In [16]:
### examine data dicts for some of the columns that could not convert
for col in step2_non_coerced_cols:
    print('-'*80)
    print(col, DD[DD.column==col].description.values[0])
    print(EST[col].value_counts(dropna=False))

--------------------------------------------------------------------------------
S1501_C02_055E Estimate!!Percent!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!Less than high school graduate
0.0      7178
-        1276
100.0     251
33.3      239
25.0      231
         ... 
99.2        1
92.9        1
87.9        1
87.7        1
86.2        1
Name: S1501_C02_055E, Length: 932, dtype: int64
--------------------------------------------------------------------------------
S1501_C02_056E Estimate!!Percent!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!High school graduate (includes equivalency)
0.0     2556
-        967
8.3      396
7.9      372
6.9      372
        ... 
88.3       1
78.8       1
77.6       1
75.3       1
78.5       1
Name: S1501_C02_056E, Length: 805, dtype: int64
------------------------------------------------------------

In [17]:
# for columns where every record is (X), replace with NaN
step3_coerced_cnt = 0
step3_total_obj_cols = 0
step3_non_coerced_cols = []
for col in EST:
    if EST.dtypes[col] == 'object' and col not in ['GEO_ID', 'NAME']:
        step3_total_obj_cols += 1
        try:
            if len(EST[col].value_counts(dropna=False)) == 1 and EST[col].values[0] == '(X)':
                new_col = pd.to_numeric(EST[col], errors='coerce')
                EST[col] = new_col
                step3_coerced_cnt += 1
            else:
                raise Exception
        except Exception as e:
            step3_non_coerced_cols.append(col)
print('{} total object columns examined in this step'.format(step3_total_obj_cols))
print('{} additional columns converted to a numeric type.'.format(step3_coerced_cnt))
print('{} columns could not be converted in this step to a numeric type.'.format(len(step3_non_coerced_cols)))

22 total object columns examined in this step
0 additional columns converted to a numeric type.
22 columns could not be converted in this step to a numeric type.


In [18]:
### examine data dicts for some of the columns that could not convert
for col in step3_non_coerced_cols:
    print('-'*80)
    print(col, DD[DD.column==col].description.values[0])
    print(EST[col].value_counts(dropna=False))

--------------------------------------------------------------------------------
S1501_C02_055E Estimate!!Percent!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!Less than high school graduate
0.0      7178
-        1276
100.0     251
33.3      239
25.0      231
         ... 
99.2        1
92.9        1
87.9        1
87.7        1
86.2        1
Name: S1501_C02_055E, Length: 932, dtype: int64
--------------------------------------------------------------------------------
S1501_C02_056E Estimate!!Percent!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!High school graduate (includes equivalency)
0.0     2556
-        967
8.3      396
7.9      372
6.9      372
        ... 
88.3       1
78.8       1
77.6       1
75.3       1
78.5       1
Name: S1501_C02_056E, Length: 805, dtype: int64
------------------------------------------------------------

In [19]:
pd.set_option('max_rows', None)
print(EST['S1501_C03_059E'].value_counts(dropna=False))
pd.set_option('max_rows', 10)

-           1251
41250         96
46250         85
36250         82
38750         81
33750         79
43750         77
45000         73
42500         72
31250         68
32500         65
28750         63
40000         61
51250         61
48750         61
47500         59
41875         57
35000         57
37500         50
30000         50
26250         48
52500         47
40625         45
23750         45
53750         43
27500         41
56250         41
50000         40
61250         40
35625         39
39375         38
31875         38
55000         38
36875         37
62500         37
63750         35
38125         34
44375         33
53125         33
57500         32
43333         32
49375         31
39583         31
37917         30
33333         30
44583         30
45625         30
42083         30
36667         30
34375         29
43125         28
50625         28
58750         28
46875         27
41667         27
25000         26
60000         26
30625         26
33125         

In [20]:
# for columns that have ','' and/or '+' in values (and optionally a '-' record), replace
# the ',' and '+', then coerce
# for columns where every record is (X), replace with NaN
step4_coerced_cnt = 0
step4_total_obj_cols = 0
step4_non_coerced_cols = []
for col in EST:
    if EST.dtypes[col] == 'object' and col not in ['GEO_ID', 'NAME']:
        step4_total_obj_cols += 1
        try:
            # replace records (not substrings) that are just dashes with placeholder number
            tmp_df = pd.DataFrame(EST[col])

            # perform the conversions
            ## replace '-' with a placeholder if val=='-'
            tmp_df.loc[tmp_df[col].str.strip() == '-', col] = -999
            ## replace '(X)' with a placeholder if val=='('X')'
            tmp_df.loc[tmp_df[col].str.strip() == '(X)', col] = -999
            # replace ',', '+', and '-' within strings with blanks
            tmp_df[col] = tmp_df[col].str.strip()\
                .str.replace(',', '')\
                .str.replace('+', '')\
                .str.replace('-', '')
            
            new_col = pd.to_numeric(tmp_df[col])
            
            # set placeholder to nan
            new_new_col = []
            for item in new_col.values:
                if item == -999:
                    new_new_col.append(np.nan)
                else:
                    new_new_col.append(item)
                    
            EST[col] = new_new_col
            step4_coerced_cnt += 1
        except Exception as e:
            print(col, e)
            step4_non_coerced_cols.append(col)
print('{} total object columns examined in this step'.format(step4_total_obj_cols))
print('{} additional columns converted to a numeric type.'.format(step4_coerced_cnt))
print('{} columns could not be converted in this step to a numeric type.'.format(len(step4_non_coerced_cols)))

<ipython-input-20-8b13ab098fee>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EST[col] = new_new_col


22 total object columns examined in this step
22 additional columns converted to a numeric type.
0 columns could not be converted in this step to a numeric type.


In [21]:
# drop records from PR to give 50 states + dc
loc_data = EST[['GEOID', 'NAME']].merge(gaz[['GEOID', 'USPS']], how='left', on='GEOID')
print(loc_data.USPS.nunique())
print(loc_data.USPS.unique())
PR_geoids = loc_data[loc_data.USPS=='PR'].GEOID.values
print(len(PR_geoids))
print(len(EST))
EST = EST[~EST.GEOID.isin(PR_geoids)].reset_index(drop=True)
print(len(EST))

52
['DE' 'NE' 'MT' 'NV' 'ID' 'IL' 'HI' 'DC' 'FL' 'GA' 'IN' 'NM' 'NJ' 'NY'
 'NH' 'TN' 'PR' 'WY' 'ND' 'SC' 'UT' 'IA' 'CO' 'VT' 'WV' 'AK' 'AZ' 'AL'
 'CA' 'ME' 'LA' 'AR' 'OK' 'OR' 'PA' 'MI' 'MN' 'KS' 'KY' 'MS' 'MD' 'MA'
 'MO' 'WI' 'OH' 'NC' 'SD' 'TX' 'RI' 'VA' 'CT' 'WA']
945
74001
73056


In [22]:
# remove columns that are completely null
null_cols = EST.isnull().all()
null_cols = null_cols[null_cols==True]
print('{} columns are completely null. They will be dropped.'.format(len(null_cols)))

print(EST.shape[1])
EST = EST.drop(columns=null_cols.index.values)
print(EST.shape[1])

0 columns are completely null. They will be dropped.
289
289


In [23]:
# clean up gaz to remove PR
print(gaz.shape[0])
gaz = gaz[gaz.USPS!='PR'].reset_index(drop=True)
print(gaz.shape[0])

74001
73056


In [24]:
# clean up data dict to remove columns that were dropped
DD = DD[DD['column'].isin(EST.columns)]
print(DD.shape[0])
DD

289


,column,description
0,GEOID,Field to Join Gazetteer Data
1,GEO_ID,id
2,NAME,Geographic Area Name
3,B02001_001E,Estimate!!Total
15,DP05_0033E,Estimate!!RACE!!Total population
...,...,...
1291,S2405_C01_003E,Estimate!!Total!!Civilian employed population ...
1297,S2405_C01_004E,Estimate!!Total!!Civilian employed population ...
1303,S2405_C01_005E,Estimate!!Total!!Civilian employed population ...
1309,S2405_C01_006E,Estimate!!Total!!Civilian employed population ...


In [25]:
len(set(gaz.GEOID).intersection(set(EST.GEOID))) # expected 73056

73056

In [26]:
EST.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73056 entries, 0 to 73055
Columns: 289 entries, GEOID to DP05_0018E
dtypes: float64(129), int64(158), object(2)
memory usage: 161.1+ MB


In [27]:
DD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 1315
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   column       289 non-null    object
 1   description  289 non-null    object
dtypes: object(2)
memory usage: 6.8+ KB


In [28]:
gaz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73056 entries, 0 to 73055
Data columns (total 8 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   USPS                                                                                                                                    73056 non-null  object 
 1   GEOID                                                                                                                                   73056 non-null  int64  
 2   ALAND                                                                                                                                   73056 non-null  int64  
 3   AWATER                                                        

In [29]:
EST.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_ESTIMATES_v4.pkl'), protocol=4)
DD.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_ESTIMATES_DD_v4.pkl'), protocol=4)
gaz.to_pickle(os.path.join(ROOT, '2018_5yr_cendatagov_GAZ_v4.pkl'), protocol=4)

In [30]:
pd.set_option('display.max_rows', None)
est_stats = EST.describe(include='all').T
est_stats['n_null'] = EST.shape[0] - est_stats['count']
est_stats['percent_null'] = est_stats.n_null / EST.shape[0] * 100.


est_stats

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,n_null,percent_null
GEOID,73056,NaN,NaN,NaN,2.7826e+10,1.58101e+10,1.00102e+09,1.21271e+10,2.7123e+10,4.1036e+10,5.6046e+10,0,0
GEO_ID,73056,73056,1400000US47151975000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
NAME,73056,73056,"Census Tract 5512.01, Los Angeles County, Cali...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
B02001_001E,73056,NaN,NaN,NaN,4419.94,2286.01,0,2912,4120.5,5537,70271,0,0
DP05_0033PE,73056,NaN,NaN,NaN,4419.94,2286.01,0,2912,4120.5,5537,70271,0,0
DP05_0034PE,72411,NaN,NaN,NaN,96.8369,3.08759,33.3,95.7,97.6,98.8,100,645,0.882884
DP05_0035PE,72411,NaN,NaN,NaN,3.16319,3.0876,0,1.2,2.4,4.3,66.7,645,0.882884
DP05_0036PE,72411,NaN,NaN,NaN,96.8369,3.08759,33.3,95.7,97.6,98.8,100,645,0.882884
DP05_0037PE,72411,NaN,NaN,NaN,72.3228,25.2919,0,60,80.9,92,100,645,0.882884
DP05_0038PE,72411,NaN,NaN,NaN,13.8416,21.8566,0,1,4.3,15.5,100,645,0.882884


In [31]:
est_stats.to_excel(os.path.join(ROOT, 'EST_v4_STATISTICS.xlsx'))

In [32]:
EST.shape

(73056, 289)

In [33]:
gaz.shape

(73056, 8)

In [34]:
DD.shape

(289, 2)